# Pre-processing solver dataset and output clean dataset which is the input for clustering.

# Table of Contents

1. [Introduction](#Introduction)
2. [Import Packages](#Import_packages)
3. [Set environment variables to retrieve data from Ceph and map to dataframe](#retrieve_data_from_Ceph)
4. [Load Error data from csv file created in the above step](#Load_data)
5. [Split the error components from the log message](#split_error)
6. [Example of error logs](#example)
7. [Prepare the data for clustering](#prepare_data)
8. [Cleaning clustering data](#clean_data)
9. [Tokenization](#tokenization)
10. [Save the cleaned data for clustering](#save_data)

## Introduction  <a id='Introduction'></a>

The purpose of this notebook is to preprocess solver data, i.e, extract error data from solver data, prepare data for clustering, clean and tokenize the clustering data and save the clean dataset for [ClusterError](./ClusterErrors.ipynb) notebook. 


Currently grafana metrics from the SolverResultsStore show over 700k results for solvers. This notebook filters for documents with solver errors. Each solver has around 20k - 30k results with solver error.\
Within the results, there are currently five different solvers:
   - solver-rhel-8-py36
   - solver-fedora-31-py38
   - solver-fedora-31-py37
   - solver-fedora-32-py38
   - solver-fedora-32-py37

## Import packages <a id='Import_packages'></a>

In [1]:
import pandas as pd
import regex as re
import pickle
import nltk

from thoth.report_processing.components.solver import Solver
from nltk.tokenize import TreebankWordTokenizer
from string import punctuation  

In [2]:
pd.set_option('max_colwidth', 1000)

## Set environment variables to retrieve data from Ceph and map to dataframe  <a id='retrieve_data_from_Ceph'></a>
### DO NOT Run this everytime. 

### All the result in solver report are described below:

- **environment,** information about the environment on which the package has being solved;
- **environment_packages,** information about external packages installed on the environment;
- **errors,** if the installation of a package was not succesfull there will be information stored for each package error;
    - **details,**
        - **command**,
        - **message**,
        - **return_code**,
        - **stderr**,
        - **stdout**,
        - **timeout**,
    - **index_url,** from where the package was download;
    - **package_name;**
    - **package_version;**
    - **is_provided_package,** flag for storing package;
    - **is_provided_package_version,** flag for storing package;
    - **type,** error type;
- **tree**, all the packages installed in the dependency tree and information about them;
    - **dependencies**
    - **metadata** of the package as taken from importlib_metadata;
    - **index_url** from where the package was download;
    - **package_name;**
    - **package_version;**
    - **sha256;**
- **unparsed** if there are packages in the tree that could not be parsed;
- **unresolved,** if there are packages in the tree that could not be solved;

In [8]:
get_fresh_data = True
limit_results = True
max_ids = 10
solver_filters = ["solver"]

In [9]:
import os
THOTH_S3_ENDPOINT_URL = os.environ['THOTH_S3_ENDPOINT_URL']
THOTH_CEPH_BUCKET = os.environ['THOTH_CEPH_BUCKET']
THOTH_CEPH_HOST = os.environ['THOTH_CEPH_HOST']
THOTH_CEPH_BUCKET_PREFIX = os.environ['THOTH_CEPH_BUCKET_PREFIX']
THOTH_DEPLOYMENT_NAME = "os.environ['THOTH_DEPLOYMENT_NAME']"

THOTH_CEPH_KEY_ID = os.environ['THOTH_CEPH_KEY_ID']
THOTH_CEPH_SECRET_KEY = os.environ['THOTH_CEPH_SECRET_KEY']

In [10]:
if get_fresh_data:
    count = 0
    # Connect to thoth storage
    from thoth.storages import SolverResultsStore
    store = SolverResultsStore(region="us-east-1")
    store.connect()
    
    solver_reports_extracted_data = []
    solver_errors = []

    # This block filters for documents with solver errors(Retrieve solver reports from Ceph and 
    # save only if it has "errors" in 
    for document_id in store.get_document_listing():
        if any(solver in document_id for solver in solver_filters):
            print(document_id)
            try:
                solver_document = store.retrieve_document(document_id=document_id)
                solver_report_extracted_data = Solver.extract_data_from_solver_metadata(solver_document["metadata"])
                errors = Solver.extract_errors_from_solver_result(solver_document["result"]["errors"])   
                for error in errors:
                    error['document_id'] = solver_report_extracted_data['document_id']
                    error['datetime'] = solver_report_extracted_data['datetime']
                    error['analyzer_version'] = solver_report_extracted_data['analyzer_version']
                    error['environment'] = solver_document["result"]["environment"]
                    solver_errors.append(error)
                count += 1
                print(count)
            except Exception as e:
                print(document_id, e)
            
        
            
    solver_total = pd.DataFrame(solver_errors).reset_index()

    solver_total['solver'] = solver_total['document_id'].str.rsplit("-", n=1).str[0]
    solver_total['solver'] = solver_total['solver'].replace('solver-rhel-8.0-py36', 'solver-rhel-8-py36')
    
    solver_total.to_csv('error_data.csv', index=False)
    print(len(solver_total))

solver-fedora-31-py37-000aa486
1
solver-fedora-31-py37-000e00a1
2
solver-fedora-31-py37-0022caa4
3
solver-fedora-31-py37-002b0f72
4
solver-fedora-31-py37-003d9de4
5
solver-fedora-31-py37-0040834f
6
solver-fedora-31-py37-004344f0
7
solver-fedora-31-py37-004b1de3
8
solver-fedora-31-py37-00500010
9
solver-fedora-31-py37-0050e909
10
solver-fedora-31-py37-005ab6b4
11
solver-fedora-31-py37-005dcfa7
12
solver-fedora-31-py37-0063867d
13
solver-fedora-31-py37-0064bc0a
14
solver-fedora-31-py37-006c96c8
15
solver-fedora-31-py37-00734e67
16
solver-fedora-31-py37-007b905b
17
solver-fedora-31-py37-007cc4aa
18
solver-fedora-31-py37-008441f2
19
solver-fedora-31-py37-0084da4c
20
solver-fedora-31-py37-00866703
21
solver-fedora-31-py37-008709c8
22
solver-fedora-31-py37-0089ebd4
23
solver-fedora-31-py37-008adcec
24
solver-fedora-31-py37-008c344c
25
solver-fedora-31-py37-008e229c
26
solver-fedora-31-py37-00912d02
27
solver-fedora-31-py37-0095681b
28
solver-fedora-31-py37-009aaefa
29
solver-fedora-31-py37-0

239
solver-fedora-31-py37-0537d96f
240
solver-fedora-31-py37-053a0dc6
241
solver-fedora-31-py37-0542bb90
242
solver-fedora-31-py37-054b93dd
243
solver-fedora-31-py37-054c29b6
244
solver-fedora-31-py37-054edf0b
245
solver-fedora-31-py37-05587cf8
246
solver-fedora-31-py37-055a5f29
247
solver-fedora-31-py37-05612f3a
248
solver-fedora-31-py37-0563aa33
249
solver-fedora-31-py37-05648c98
250
solver-fedora-31-py37-0578c8c9
251
solver-fedora-31-py37-057c6e11
252
solver-fedora-31-py37-057f1ef8
253
solver-fedora-31-py37-0583767a
254
solver-fedora-31-py37-058797d6
255
solver-fedora-31-py37-05899d67
256
solver-fedora-31-py37-058a029c
257
solver-fedora-31-py37-0591fe88
258
solver-fedora-31-py37-0593451c
259
solver-fedora-31-py37-05954397
260
solver-fedora-31-py37-05983e12
261
solver-fedora-31-py37-059882c3
262
solver-fedora-31-py37-05a01f95
263
solver-fedora-31-py37-05a120b0
264
solver-fedora-31-py37-05b796e6
265
solver-fedora-31-py37-05bb5411
266
solver-fedora-31-py37-05c347dc
267
solver-fedora-31

474
solver-fedora-31-py37-0a3fea82
475
solver-fedora-31-py37-0a4827c1
476
solver-fedora-31-py37-0a4d5987
477
solver-fedora-31-py37-0a59f5f6
478
solver-fedora-31-py37-0a641f1d
479
solver-fedora-31-py37-0a6a1a65
480
solver-fedora-31-py37-0a6c4d86
481
solver-fedora-31-py37-0a6ebd28
482
solver-fedora-31-py37-0a77618c
483
solver-fedora-31-py37-0a847f92
484
solver-fedora-31-py37-0a8b6b3a
485
solver-fedora-31-py37-0a8da0db
486
solver-fedora-31-py37-0a8e34d6
487
solver-fedora-31-py37-0a909cd2
488
solver-fedora-31-py37-0a98a55f
489
solver-fedora-31-py37-0a9a4d63
490
solver-fedora-31-py37-0a9fe424
491
solver-fedora-31-py37-0aa2e3d2
492
solver-fedora-31-py37-0aa33d46
493
solver-fedora-31-py37-0aa6d7c3
494
solver-fedora-31-py37-0aa9de81
495
solver-fedora-31-py37-0ab43678
496
solver-fedora-31-py37-0aba3930
497
solver-fedora-31-py37-0ac53124
498
solver-fedora-31-py37-0ac954e7
499
solver-fedora-31-py37-0ad0fef3
500
solver-fedora-31-py37-0ad13a15
501
solver-fedora-31-py37-0aeb0744
502
solver-fedora-31

709
solver-fedora-31-py37-0f1dcc05
710
solver-fedora-31-py37-0f21af7c
711
solver-fedora-31-py37-0f22526f
712
solver-fedora-31-py37-0f253d40
713
solver-fedora-31-py37-0f29edea
714
solver-fedora-31-py37-0f379657
715
solver-fedora-31-py37-0f3a49e4
716
solver-fedora-31-py37-0f4341a8
717
solver-fedora-31-py37-0f4a40be
718
solver-fedora-31-py37-0f4a95b3
719
solver-fedora-31-py37-0f569739
720
solver-fedora-31-py37-0f5d0c1c
721
solver-fedora-31-py37-0f619a4b
722
solver-fedora-31-py37-0f64be6e
723
solver-fedora-31-py37-0f684598
724
solver-fedora-31-py37-0f6b654f
725
solver-fedora-31-py37-0f6cda64
726
solver-fedora-31-py37-0f713677
727
solver-fedora-31-py37-0f89f8cb
728
solver-fedora-31-py37-0f8d1067
729
solver-fedora-31-py37-0f8d330a
730
solver-fedora-31-py37-0f93a017
731
solver-fedora-31-py37-0f96f7c2
732
solver-fedora-31-py37-0f9a8f79
733
solver-fedora-31-py37-0f9f2007
734
solver-fedora-31-py37-0fa10cba
735
solver-fedora-31-py37-0fc3f50b
736
solver-fedora-31-py37-0fca16c7
737
solver-fedora-31

945
solver-fedora-31-py37-13ede625
946
solver-fedora-31-py37-13f7fb47
947
solver-fedora-31-py37-13fa6e9f
948
solver-fedora-31-py37-13fb43d7
949
solver-fedora-31-py37-13fc2732
950
solver-fedora-31-py37-13fdc28b
951
solver-fedora-31-py37-13fef232
952
solver-fedora-31-py37-1401dcfb
953
solver-fedora-31-py37-1404b828
954
solver-fedora-31-py37-1404f9fd
955
solver-fedora-31-py37-14172f99
956
solver-fedora-31-py37-14174e91
957
solver-fedora-31-py37-141b9c67
958
solver-fedora-31-py37-141f00c7
959
solver-fedora-31-py37-141fb4e1
960
solver-fedora-31-py37-14214839
961
solver-fedora-31-py37-1424402f
962
solver-fedora-31-py37-1427d4a9
963
solver-fedora-31-py37-142c6338
964
solver-fedora-31-py37-1435ae10
965
solver-fedora-31-py37-1438e65b
966
solver-fedora-31-py37-1441014d
967
solver-fedora-31-py37-144c0496
968
solver-fedora-31-py37-1460e168
969
solver-fedora-31-py37-14696b90
970
solver-fedora-31-py37-146dd6f4
971
solver-fedora-31-py37-1477ee19
972
solver-fedora-31-py37-14788678
973
solver-fedora-31

...

60106
solver-rhel-8-py36-ef43dec5
60107
solver-rhel-8-py36-ef48b321
60108
solver-rhel-8-py36-ef4acf55
60109
solver-rhel-8-py36-ef4c2917
60110
solver-rhel-8-py36-ef4d23d0
60111
solver-rhel-8-py36-ef4ea357
60112
solver-rhel-8-py36-ef512f93
60113
solver-rhel-8-py36-ef51efa6
60114
solver-rhel-8-py36-ef604353
60115
solver-rhel-8-py36-ef6388bb
60116
solver-rhel-8-py36-ef640430
60117
solver-rhel-8-py36-ef72047c
60118
solver-rhel-8-py36-ef7288f7
60119
solver-rhel-8-py36-ef75c1c1
60120
solver-rhel-8-py36-ef75dd84
60121
solver-rhel-8-py36-ef77022c
60122
solver-rhel-8-py36-ef79c275
60123
solver-rhel-8-py36-ef7da5d8
60124
solver-rhel-8-py36-ef7fc132
60125
solver-rhel-8-py36-ef8bbc9b
60126
solver-rhel-8-py36-ef9108a4
60127
solver-rhel-8-py36-ef93f914
60128
solver-rhel-8-py36-ef964b51
60129
solver-rhel-8-py36-ef98beac
60130
solver-rhel-8-py36-ef9c40e1
60131
solver-rhel-8-py36-efa1ea34
60132
solver-rhel-8-py36-efa2d999
60133
solver-rhel-8-py36-efa32e39
60134
solver-rhel-8-py36-efa77198
60135
solver-r

60348
solver-rhel-8-py36-f2d96b21
60349
solver-rhel-8-py36-f2debf3c
60350
solver-rhel-8-py36-f2e3486e
60351
solver-rhel-8-py36-f2e73dff
60352
solver-rhel-8-py36-f2e819ef
60353
solver-rhel-8-py36-f2e8b65e
60354
solver-rhel-8-py36-f2e9ed7a
60355
solver-rhel-8-py36-f2efca65
60356
solver-rhel-8-py36-f2f1ab4e
60357
solver-rhel-8-py36-f2fe16ba
60358
solver-rhel-8-py36-f30221d4
60359
solver-rhel-8-py36-f305acb2
60360
solver-rhel-8-py36-f30e7809
60361
solver-rhel-8-py36-f3118aff
60362
solver-rhel-8-py36-f313c661
60363
solver-rhel-8-py36-f315f281
60364
solver-rhel-8-py36-f3169242
60365
solver-rhel-8-py36-f31ab743
60366
solver-rhel-8-py36-f31d7649
60367
solver-rhel-8-py36-f31f2e6e
60368
solver-rhel-8-py36-f31f7abd
60369
solver-rhel-8-py36-f320bc3f
60370
solver-rhel-8-py36-f3232920
60371
solver-rhel-8-py36-f323c32c
60372
solver-rhel-8-py36-f3246b35
60373
solver-rhel-8-py36-f326a9f8
60374
solver-rhel-8-py36-f32b3569
60375
solver-rhel-8-py36-f32c1973
60376
solver-rhel-8-py36-f3325d7b
60377
solver-r

60589
solver-rhel-8-py36-f6245642
60590
solver-rhel-8-py36-f628296a
60591
solver-rhel-8-py36-f6289995
60592
solver-rhel-8-py36-f628d046
60593
solver-rhel-8-py36-f629cebb
60594
solver-rhel-8-py36-f62fd973
60595
solver-rhel-8-py36-f630e53c
60596
solver-rhel-8-py36-f63ba3d0
60597
solver-rhel-8-py36-f63f3e2d
60598
solver-rhel-8-py36-f6400012
60599
solver-rhel-8-py36-f640d142
60600
solver-rhel-8-py36-f6459bfd
60601
solver-rhel-8-py36-f64695de
60602
solver-rhel-8-py36-f6473ea5
60603
solver-rhel-8-py36-f6474ed0
60604
solver-rhel-8-py36-f64bb42f
60605
solver-rhel-8-py36-f6512fa6
60606
solver-rhel-8-py36-f653c5e0
60607
solver-rhel-8-py36-f6567cb9
60608
solver-rhel-8-py36-f657f82e
60609
solver-rhel-8-py36-f662717f
60610
solver-rhel-8-py36-f66c8c4d
60611
solver-rhel-8-py36-f66cf21b
60612
solver-rhel-8-py36-f66f262a
60613
solver-rhel-8-py36-f67054e0
60614
solver-rhel-8-py36-f67e6a11
60615
solver-rhel-8-py36-f6807719
60616
solver-rhel-8-py36-f6846cc0
60617
solver-rhel-8-py36-f686589f
60618
solver-r

60830
solver-rhel-8-py36-f9822513
60831
solver-rhel-8-py36-f98a9ab1
60832
solver-rhel-8-py36-f98b93d3
60833
solver-rhel-8-py36-f9918f24
60834
solver-rhel-8-py36-f9963a18
60835
solver-rhel-8-py36-f997be77
60836
solver-rhel-8-py36-f99d17bb
60837
solver-rhel-8-py36-f9a8ae87
60838
solver-rhel-8-py36-f9aaef0e
60839
solver-rhel-8-py36-f9ac25eb
60840
solver-rhel-8-py36-f9ad4bd6
60841
solver-rhel-8-py36-f9adc80f
60842
solver-rhel-8-py36-f9b7043e
60843
solver-rhel-8-py36-f9b93a0d
60844
solver-rhel-8-py36-f9bd5dfb
60845
solver-rhel-8-py36-f9bda5ad
60846
solver-rhel-8-py36-f9bdf8e1
60847
solver-rhel-8-py36-f9c0e102
60848
solver-rhel-8-py36-f9c0e826
60849
solver-rhel-8-py36-f9c1ac30
60850
solver-rhel-8-py36-f9c1e1ff
60851
solver-rhel-8-py36-f9c272d3
60852
solver-rhel-8-py36-f9c2ef82
60853
solver-rhel-8-py36-f9c64b5a
60854
solver-rhel-8-py36-f9c6c62f
60855
solver-rhel-8-py36-f9c6f279
60856
solver-rhel-8-py36-f9cbd38d
60857
solver-rhel-8-py36-f9ccc7d4
60858
solver-rhel-8-py36-f9cdcdc9
60859
solver-r

61071
solver-rhel-8-py36-fcbb52bd
61072
solver-rhel-8-py36-fcbc7734
61073
solver-rhel-8-py36-fcc9c131
61074
solver-rhel-8-py36-fcccdf8f
61075
solver-rhel-8-py36-fcd0f3e7
61076
solver-rhel-8-py36-fcd2fa79
61077
solver-rhel-8-py36-fcd5652f
61078
solver-rhel-8-py36-fce195a4
61079
solver-rhel-8-py36-fce3bf04
61080
solver-rhel-8-py36-fce40a71
61081
solver-rhel-8-py36-fce446df
61082
solver-rhel-8-py36-fce5dc03
61083
solver-rhel-8-py36-fce6620c
61084
solver-rhel-8-py36-fce69a18
61085
solver-rhel-8-py36-fce9342c
61086
solver-rhel-8-py36-fcec6839
61087
solver-rhel-8-py36-fcecd8d2
61088
solver-rhel-8-py36-fcf0eb4d
61089
solver-rhel-8-py36-fcf29327
61090
solver-rhel-8-py36-fcf38888
61091
solver-rhel-8-py36-fcf535ab
61092
solver-rhel-8-py36-fcf7990d
61093
solver-rhel-8-py36-fd02007f
61094
solver-rhel-8-py36-fd02e2be
61095
solver-rhel-8-py36-fd067c67
61096
solver-rhel-8-py36-fd0b4810
61097
solver-rhel-8-py36-fd1181c4
61098
solver-rhel-8-py36-fd148e8b
61099
solver-rhel-8-py36-fd167bca
61100
solver-r

## Load Error data from csv file created in the above step <a id='Load_data'></a>

In [11]:
if get_fresh_data:
    solver_total_errors_df = solver_total
else:
    solver_total_errors_df = pd.read_csv('error_data.csv')

In [12]:
solver_total_errors_df['solver'] = solver_total_errors_df['solver'].replace('solver-rhel-8.0-py36', 'solver-rhel-8-py36')

In [13]:
solver_total_errors_df['solver'].unique()

array(['solver-fedora-31-py37', 'solver-fedora-31-py38',
       'solver-fedora-32-py37', 'solver-fedora-32-py38',
       'solver-rhel-8-py36'], dtype=object)

In [14]:
solver_total_errors_df.head(5)

,index,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout,document_id,datetime,analyzer_version,environment,solver
0,0,Cython,0.23.4,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps Cython===0.23.4 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-9eme38jp\n cwd: /tmp/pip-install-bnnj141r/Cython/\n Complete output (343 lines):\n Unable to find pgen, not compiling formal grammar.\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n copying cython.py -> build/lib.linux-x86_64-3.7\n creating build/lib.linux-x86_64-3.7/Cython\n copying Cython/__init__.py -> build/lib.linux-x86_64-3.7/Cython\n copying Cython/Utils.py -> build/lib.linux-x86_64-3.7/Cython\n copying ...",1.0,"ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-9eme38jp\n cwd: /tmp/pip-install-bnnj141r/Cython/\n Complete output (343 lines):\n Unable to find pgen, not compiling formal grammar.\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n copying cython.py -> build/lib.linux-x86_64-3.7\n creating build/lib.linux-x86_64-3.7/Cython\n copying Cython/__init__.py -> build/lib.linux-x86_64-3.7/Cython\n copying Cython/Utils.py -> build/lib.linux-x86_64-3.7/Cython\n copying Cython/TestUtils.py -> build/lib.linux-x86_64-...",Collecting Cython===0.23.4\n Downloading Cython-0.23.4.tar.gz (1.6 MB)\nBuilding wheels for collected packages: Cython\n Building wheel for Cython (setup.py): started\n Building wheel for Cython (setup.py): finished with status 'error'\n Running setup.py clean for Cython\nFailed to build Cython\nInstalling collected packages: Cython\n Running setup.py install for Cython: started\n Running setup.py install for Cython: finished with status 'error'\n,60.0,solver-fedora-31-py37-0022caa4,2020-08-05T21:54:08.679464,1.6.0,"{'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '4.18.0-147.8.1.el8_1.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Wed Feb 26 03:08:15 UTC 2020', 'python_full_version': '3.7.7', 'python_version': '3.7', 'sys_platform': 'linux'}",solver-fedora-31-py37
1,1,word2vec,0.7.1,https://pypi.org/simple,command_error,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps word2vec===0.7.1 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org","Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-zim3n1sc/word2vec/setup.py'""'""'; __file__='""'""'/tmp/pip-install-zim3n1sc/word2vec/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(co

## Split the error components from the log message <a id='split_error'></a>

In [15]:
def split_log(log_messages):
    log_messages = log_messages.split('\n')
    return log_messages

In [16]:
error_df = pd.DataFrame()
error_df['index'] = solver_total_errors_df['index']
error_df['document_id'] = solver_total_errors_df['document_id']
error_df['command'] = solver_total_errors_df['command']
error_df['package_name'] = solver_total_errors_df['package_name']
error_df['package_version'] = solver_total_errors_df['package_version']
error_df['solver'] = solver_total_errors_df['solver']
error_df['datetime'] = solver_total_errors_df['datetime']
error_df['environment'] = solver_total_errors_df['environment']
error_df['analyzer_version'] = solver_total_errors_df['analyzer_version']

In [17]:
error_df['message'] = solver_total_errors_df['message']
error_df['split_message']= solver_total_errors_df.apply(lambda row: split_log(row.message),axis = 1)

### Example of error logs: <a id='example'></a>

In [18]:
solver_total_errors_df['message'][1]

'Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:\n     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-zim3n1sc/word2vec/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-zim3n1sc/word2vec/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-pip-egg-info-hv7fcspg\n         cwd: /tmp/pip-install-zim3n1sc/word2vec/\n    Complete output (6 lines):\n    Traceback (most recent call last):\n      File "<string>", line 1, in <module>\n      File "/tmp/pip-install-zim3n1sc/word2vec/setup.py", line 29\n        print \' \'.join(command)\n                ^\n    SyntaxError: invalid syntax\n    ----------------------------------------\nERROR: Command errored out with exit status 1: python setup.py egg

In [19]:
error_df['split_message'][1]

['Command exited with non-zero status code (1):     ERROR: Command errored out with exit status 1:',
 '     command: /home/solver/venv/bin/python3 -c \'import sys, setuptools, tokenize; sys.argv[0] = \'"\'"\'/tmp/pip-install-zim3n1sc/word2vec/setup.py\'"\'"\'; __file__=\'"\'"\'/tmp/pip-install-zim3n1sc/word2vec/setup.py\'"\'"\';f=getattr(tokenize, \'"\'"\'open\'"\'"\', open)(__file__);code=f.read().replace(\'"\'"\'\\r\\n\'"\'"\', \'"\'"\'\\n\'"\'"\');f.close();exec(compile(code, __file__, \'"\'"\'exec\'"\'"\'))\' egg_info --egg-base /tmp/pip-pip-egg-info-hv7fcspg',
 '         cwd: /tmp/pip-install-zim3n1sc/word2vec/',
 '    Complete output (6 lines):',
 '    Traceback (most recent call last):',
 '      File "<string>", line 1, in <module>',
 '      File "/tmp/pip-install-zim3n1sc/word2vec/setup.py", line 29',
 "        print ' '.join(command)",
 '                ^',
 '    SyntaxError: invalid syntax',
 '    ----------------------------------------',
 'ERROR: Command errored out with ex

In [20]:
solver_total_errors_df['message'][19]

"Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement paddlepaddle===1.0.2 (from versions: 1.2.0, 1.2.1, 1.3.0, 1.3.1, 1.3.2, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.5.2, 1.6.0rc0, 1.6.0, 1.6.1, 1.6.2, 1.6.3, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.8.3, 2.0.0a0)\nERROR: No matching distribution found for paddlepaddle===1.0.2\nWARNING: You are using pip version 20.1.1; however, version 20.2.1 is available.\nYou should consider upgrading via the '/home/solver/venv/bin/python3 -m pip install --upgrade pip' command.\n"

In [21]:
error_df['split_message'][19]

['Command exited with non-zero status code (1): ERROR: Could not find a version that satisfies the requirement paddlepaddle===1.0.2 (from versions: 1.2.0, 1.2.1, 1.3.0, 1.3.1, 1.3.2, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.5.2, 1.6.0rc0, 1.6.0, 1.6.1, 1.6.2, 1.6.3, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.8.3, 2.0.0a0)',
 'ERROR: No matching distribution found for paddlepaddle===1.0.2',
 'WARNING: You are using pip version 20.1.1; however, version 20.2.1 is available.',
 "You should consider upgrading via the '/home/solver/venv/bin/python3 -m pip install --upgrade pip' command.",
 '']

In [22]:
solver_total_errors_df['message'][226]

'Failed to successfully execute function in Python interpreter: Traceback (most recent call last):\n  File "<string>", line 21, in <module>\n  File "<string>", line 8, in _find_distribution_name\n  File "/home/solver/venv/lib/python3.7/site-packages/pkg_resources.py", line 1545, in <module>\n    register_loader_type(importlib_bootstrap.SourceFileLoader, DefaultProvider)\nAttributeError: module \'importlib._bootstrap\' has no attribute \'SourceFileLoader\'\n'

In [23]:
error_df['split_message'][226]

['Failed to successfully execute function in Python interpreter: Traceback (most recent call last):',
 '  File "<string>", line 21, in <module>',
 '  File "<string>", line 8, in _find_distribution_name',
 '  File "/home/solver/venv/lib/python3.7/site-packages/pkg_resources.py", line 1545, in <module>',
 '    register_loader_type(importlib_bootstrap.SourceFileLoader, DefaultProvider)',
 "AttributeError: module 'importlib._bootstrap' has no attribute 'SourceFileLoader'",
 '']

In [24]:
def split_error_components(log_messages):
    ids_with_different_log_pattern = []
    Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception = {}, {}, {}, {}, {}, {}, {}
    for idx, msg in enumerate(log_messages):
        msg = msg.replace("Error:\n", "Error:")
        sentences = [x.strip() for x in msg.split('\n')]
        for id, sent in enumerate(sentences):
            if re.match(r".*WARNING.*|.*warnings.*|.*except .*|^copying.*|^checking.*", sent):
                pass
            elif re.match(r"^command.*", sent):
                if idx in command_info.keys() and sent not in command_info[idx]:
                    command_info[idx].append(sent)
                else:
                    command_info[idx] = [sent]
            elif re.match(r"^cwd.*", sent):
                if idx in cwd.keys() and sent not in cwd[idx]:
                    cwd[idx].append(sent)
                else:
                    cwd[idx] = [sent]
            elif re.match(r"^Complete output.*", sent):
                number_of_lines = re.findall(r'\d+', sent) 
                if idx in Complete_output.keys() and sent not in Complete_output[idx]:
                    pass
                else:
                    Complete_output[idx] = sentences[id:id+int(number_of_lines[0])+1]
                    for txt in Complete_output[idx]:
                        if re.match(r"^Exception.*", txt):
                            exception[idx] = [txt]
            elif re.match(r".*unable to execute.*", sent):
                specific_error[idx] = [sent]
            elif re.compile("(\w\w*Error)").findall(sent):
                if re.match('^from .* import .*', sent):
                    pass
                elif idx in specific_error.keys():
                    if sent in specific_error[idx]:
                        pass
                    if not re.match(r".*unable to execute.*", specific_error[idx][0]):
                        specific_error[idx].extend([sent])
                else:
                    specific_error[idx] = [sent]
            elif re.match(r"^Error.*", sent):
                if idx not in specific_error.keys():
                    specific_error[idx] = [sent]
            elif re.match(r"^ERROR: .*", sent):
                if idx in ERROR.keys() and sent not in ERROR[idx]:
                    if not re.match(r".*Failed.*", ERROR[idx][0]):
                        ERROR[idx].extend([sent])
                else:
                    ERROR[idx] = [sent]   
            elif re.match(r"^Command.*", sent):
                Error_info[idx] = sent
    return Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception

print(len(solver_total_errors_df['message']))

19219


In [25]:
Error_info, command_info, cwd, Complete_output, ERROR, specific_error, exception = split_error_components(solver_total_errors_df['message'])
print(len(Error_info), len(command_info), len(cwd), len(Complete_output), len(ERROR), len(specific_error), len(exception))

10401 7077 7077 7077 10295 14995 387


In [26]:
error_df['Error_info']= error_df['index'].map(Error_info)
error_df['command_info']= error_df['index'].map(command_info)
error_df['cwd']= error_df['index'].map(cwd)
error_df['Complete_output']= error_df['index'].map(Complete_output)
error_df['ERROR']= error_df['index'].map(ERROR)
error_df['Exception']= error_df['index'].map(exception)
error_df['specific_error']= error_df['index'].map(specific_error)

In [27]:
error_df.head(10)

,index,document_id,command,package_name,package_version,solver,datetime,environment,analyzer_version,message,split_message,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error
0,0,solver-fedora-31-py37-0022caa4,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps Cython===0.23.4 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",Cython,0.23.4,solver-fedora-31-py37,2020-08-05T21:54:08.679464,"{'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '4.18.0-147.8.1.el8_1.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Wed Feb 26 03:08:15 UTC 2020', 'python_full_version': '3.7.7', 'python_version': '3.7', 'sys_platform': 'linux'}",1.6.0,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-9eme38jp\n cwd: /tmp/pip-install-bnnj141r/Cython/\n Complete output (343 lines):\n Unable to find pgen, not compiling formal grammar.\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n copying cython.py -> build/lib.linux-x86_64-3.7\n creating build/lib.linux-x86_64-3.7/Cython\n copying Cython/__init__.py -> build/lib.linux-x86_64-3.7/Cython\n copying Cython/Utils.py -> build/lib.linux-x86_64-3.7/Cython\n copying ...","[Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:, command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-9eme38jp, cwd: /tmp/pip-install-bnnj141r/Cython/, Complete output (343 lines):, Unable to find pgen, not compiling formal grammar., running bdist_wheel, running build, running build_py, creating build, creating build/lib.linux-x86_64-3.7, copying cython.py -> build/lib.linux-x86_64-3.7, creating build/lib.linux-x86_64-3.7/Cython, copying Cython/__init__.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/Utils.py -> build/lib.linux-x86_64-3.7/Cython, copying...",Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"[command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-9eme38jp, command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' install --record /tmp/pip-record-d45ftp_t/install-record.txt --single-version-externally-managed --compile --install-headers /home/solver/venv/include/site/python3.7/Cython]",[cwd: /tmp/pip-install-bnnj141r/Cython/],"[Complete

## Prepare the data for clustering <a id='prepare_data'></a>

Look for data in the 'specific_error' column of the dataframe. If not present, look at 'ERROR' column or 'Error_info' column. If all the extracted columns are empty, take the error data from 'message' column. 

In [28]:
clustering_data = {}
context_message = {}
for idx, error in enumerate(error_df['specific_error']):
    if type(error) != list:
        #if type(error_df.iloc[idx]['Exception']) == list:
            #clustering_data[idx] = error_df.iloc[idx]['Exception']
        if pd.isnull(error) and  type(error_df.iloc[idx]['ERROR']) == list:
            context_message[idx] = error_df.iloc[idx]['ERROR']
            r = re.compile(".*Failed.*|.*CUDA.*")
            match = list(filter(r.match, error_df.iloc[idx]['ERROR']))    
            if len(error_df.iloc[idx]['ERROR']) == 1 or not match:
                clustering_data[idx] = [error_df.iloc[idx]['ERROR'][0]]
            else:
                clustering_data[idx] = match
        elif type(error_df.iloc[idx]['ERROR']) != list:
            if pd.isnull(error_df.iloc[idx]['ERROR']):
                context_message[idx] = error_df.iloc[idx]['Error_info']
                clustering_data[idx] = [error_df.iloc[idx]['Error_info']]
                if type(error_df.iloc[idx]['Error_info']) != list:
                    if pd.isnull(error_df.iloc[idx]['Error_info']):
                        context_message[idx] = error_df.iloc[idx]['message']
                        clustering_data[idx] = [error_df.iloc[idx]['message']]        
    else:
        context_message[idx] = error_df.iloc[idx]['specific_error']     
        # Extracting just the words with 'Error' as suffix to it and handling some edge cases. 
        r = re.compile("(\w\w*Error)")     
        string = ' '.join(error)
        match = re.findall(r, string)
        if match:
            if len(match) == 1:
                clustering_data[idx] = match
            else:
                if 'SyntaxError' in match:
                    for err in match:
                        if re.match(rf'.*{err},|.*{err}\),', string):
                            clustering_data[idx] = ['SyntaxError']
                            break
                    if idx not in clustering_data.keys():
                        clustering_data[idx] = list(set(match))
                elif {'OSError', 'EnvironmentError'} == set(match):
                    clustering_data[idx] = ['OSError']
                elif {'Error_GetLastError', 'AttributeError'} == set(match):
                    clustering_data[idx] = ['AttributeError']
                elif {'GrabNetworkError', 'GrabError', 'ModuleNotFoundError'} == set(match):
                    clustering_data[idx] = ['ModuleNotFoundError']
                elif {'ReadTimeoutError', 'EnvironmentError'} == set(match):
                    clustering_data[idx] = ['ReadTimeoutError']
                elif {'NewConnectionError', 'EnvironmentError'} == set(match):
                    clustering_data[idx] = ['NewConnectionError']
                else:
                    clustering_data[idx] = list(set(match))
        else:
            clustering_data[idx] = error
    if clustering_data[idx] == ['Command exited with non-zero status code (-9):']:
        clustering_data[idx] = ['Error Info not Available']
    string = ' '.join(clustering_data[idx])
    if re.match(".*make: .*|.*CMake .*|.*zError.*|.*SWIG_Python_TypeError.*|.*CatchlibLZMAError.*|.*PyExc_MemoryError.*|.*ViZDoomError.*|.*PyExc_KeyError.*|.*icsneoGetError.*|.*LapackSrcNotFoundError.*|.*PyFrame_FastToLocalsWithError.*", string):
    #if re.match(".*zError.*|.*SWIG_Python_TypeError.*|.*CatchlibLZMAError.*|.*PyExc_KeyError.*|.*PyFrame_FastToLocalsWithError.*", clustering_data[idx][0]):
        clustering_data[idx] = error_df.iloc[idx]['ERROR']

In [29]:
len(clustering_data), len(context_message)

(19219, 19219)

## Cleaning clustering data <a id='clean_data'></a>

In [30]:
_line_number = r'(at line[:]*\s*\d+)'
_url = r'(http[s]|root|srm|file)*:(//|/)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
_filepath = "(/[a-zA-Z\./]*[\s]?)"
path_regex = re.compile(r'(\b\w+://)\S+(?=\s)')
file_regex = re.compile(r'(\b[f|F]ile( exists)?:?\s?)/\S+(?=\s)')
py_regex = re.compile(r'/?\b[-./_a-zA-Z0-9]+\.py\b')
long_regex = re.compile(r'[-/_a-zA-Z0-9]{25,}')

In [31]:
def remove_whitespaces(sentence):
    return " ".join(sentence.split())

def substitute_path(string):
    string = path_regex.sub(r'\1', string)
    string = py_regex.sub(r' ', string)
    string = file_regex.sub(r'\1', string)
    string = long_regex.sub(r'', string)
    return string

def cleaner(log_messages):
    clean_log = {}
    for key in log_messages:
        for item in log_messages[key]:
            item = re.sub(_line_number, "at line *", item)
            item = re.sub(_url, " ", item)
            item = re.sub(_filepath, " ", item)
            item = substitute_path(item)
            if key in clean_log.keys() and item not in clean_log[key]:
                clean_log[key] = clean_log[key] + ' ' + remove_whitespaces(item)
            else:
                clean_log[key] = remove_whitespaces(item)
    return clean_log

In [32]:
clustering_data = cleaner(clustering_data)

In [33]:
len(clustering_data)

19219

## Tokenization <a id='tokenization'></a>

In [34]:
stemmer = nltk.PorterStemmer()
stop = punctuation + "``" + "''" + '""' + "/"
table = str.maketrans(stop, ' '*len(stop))

In [35]:
def tokenization(log_messages):
    tokenized = []
    for key, item in log_messages.items():
        item = item.replace(' Error','Error').strip()
        item = item.replace('Errno','').strip()
        item = item.replace('Error :','Error:').strip()
        item = item.replace('Exception:','').strip()
        item = item.replace('Exception','').strip()
        item = item.replace('\'','').strip()
        item = item.replace('=',' ').strip()
        if error_df['package_name'][key] in item.split():
            item = item.replace(error_df['package_name'][key],'').strip()
        if "interpreter:" in item.split():
            item = item.split("interpreter",1)[1]
        if " Please" in item:
            item = item.split("Please",1)[0]
        if "for"  in item.split():
            item = item.split("for",1)[0] 
        if "in" in item.split():
            if "in an" in item:
                item = item.split("in an",1)[1]
            else:
                item = item.split("in",1)[0]
        if "on" in item.split():
            item = re.split(" on ",item)[0] 
        if re.match(r"^ERROR:.* |^Error:.* |^error:.*", item):
            item = re.split("^ERROR: |^error: |^Error: |^error:Error:",item)[1].split('.')[0]
        if item.strip() == '':
            item = error_df.iloc[idx]['ERROR'][0].translate(table).strip()
        if "Command" in item:
            if "ERROR:" in item:
                item = item.split("ERROR:",1)[1] 
            if re.match(r".*Command errored out with exit status.*", item):
                item = "Check the logs"
        if "not found" in item:
            words = [stemmer.stem(word) for word in item.split()] 
            item = ''
            for word in words:
                item += word[0].upper() + word[1:]
            item += "Error"
        if ":" in item and item[0] != ":":
            item = item.split(":",1)[0] 
        if "," in item:
            item = item.split(",",1)[0] 
        if "JAVA_HOME" in item:
            item = 'JAVA HOME not set to a path containing the JDK'
        if re.match("(\w\w* error)", item):
            item = item.split()[0] + "Error"
        if "XML" in item:
            item = "cannot get pre-processor and compiler flags"
        item = item.translate(table).strip()
        tokenized.append(TreebankWordTokenizer().tokenize(item))
    cleaned_tokens = []
    for id, row in enumerate(tokenized):
        cleaned_tokens.append(list(filter(None, [i
                                                 for i in row 
                                                 if i != error_df['package_name'][id]
                                                 and not i.lower().isnumeric()])))
    return cleaned_tokens

In [36]:
tokenized_clustering_data = tokenization(clustering_data)

In [37]:
tokenized_clustering_data

[['SyntaxError'],
 ['SyntaxError'],
 ['FileNotFoundError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['No', 'matching', 'distribution', 'found'],
 ['TypeError'],
 ['AttributeError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['unable', 'to', 'execute', 'gcc'],
 ['ModuleNotFoundError'],
 ['ModuleNotFoundError'],
 ['ModuleNotFoundError'],
 ['FileNotFoundError'],
 ['No', 'matching', 'distribution', 'found'],
 ['ModuleNotFoundError'],
 ['ModuleNotFoundError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['SyntaxError'],
 ['ModuleNotFoundError'],
 ['No', 'matching', 'distribution', 'found'],
 ['SyntaxError'],
 ['No', 'matching', 'distribution', 'found'],
 ['ModuleNotFoundError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['unable', 'to', 'execute', 'gcc'],
 ['AttributeError'],
 ['SyntaxError'],
 ['unable', 'to', 'execute', 'gcc'],
 ['No', 'matching', 'distribution', 'found'],
 ['Check', 'the', 'logs'],
 ['AttributeError'],
 ['ModuleNotFoundError'],
 ['ValueError'],
 ['ModuleNotFoundError'],
 ['AttributeErro

## Save the cleaned data for clustering <a id='save_data'></a>

In [38]:
error_df['clustering_data'] = error_df['index'].map(clustering_data)

In [39]:
error_df['tokenized_clustering_data'] = tokenized_clustering_data

In [40]:
error_df['context_message'] = error_df['index'].map(context_message)

In [41]:
error_df.head(20)

,index,document_id,command,package_name,package_version,solver,datetime,environment,analyzer_version,message,...,Error_info,command_info,cwd,Complete_output,ERROR,Exception,specific_error,clustering_data,tokenized_clustering_data,context_message
0,0,solver-fedora-31-py37-0022caa4,"/home/solver/venv/bin/python3 -m pip install --force-reinstall --no-cache-dir --no-deps Cython===0.23.4 --index-url ""https://pypi.org/simple"" --trusted-host pypi.org",Cython,0.23.4,solver-fedora-31-py37,2020-08-05T21:54:08.679464,"{'implementation_name': 'cpython', 'implementation_version': '3.7.7', 'os_name': 'posix', 'platform_machine': 'x86_64', 'platform_python_implementation': 'CPython', 'platform_release': '4.18.0-147.8.1.el8_1.x86_64', 'platform_system': 'Linux', 'platform_version': '#1 SMP Wed Feb 26 03:08:15 UTC 2020', 'python_full_version': '3.7.7', 'python_version': '3.7', 'sys_platform': 'linux'}",1.6.0,"Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:\n command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-9eme38jp\n cwd: /tmp/pip-install-bnnj141r/Cython/\n Complete output (343 lines):\n Unable to find pgen, not compiling formal grammar.\n running bdist_wheel\n running build\n running build_py\n creating build\n creating build/lib.linux-x86_64-3.7\n copying cython.py -> build/lib.linux-x86_64-3.7\n creating build/lib.linux-x86_64-3.7/Cython\n copying Cython/__init__.py -> build/lib.linux-x86_64-3.7/Cython\n copying Cython/Utils.py -> build/lib.linux-x86_64-3.7/Cython\n copying ...",...,Command exited with non-zero status code (1): ERROR: Command errored out with exit status 1:,"[command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' bdist_wheel -d /tmp/pip-wheel-9eme38jp, command: /home/solver/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""'; __file__='""'""'/tmp/pip-install-bnnj141r/Cython/setup.py'""'""';f=getattr(tokenize, '""'""'open'""'""', open)(__file__);code=f.read().replace('""'""'\r\n'""'""', '""'""'\n'""'""');f.close();exec(compile(code, __file__, '""'""'exec'""'""'))' install --record /tmp/pip-record-d45ftp_t/install-record.txt --single-version-externally-managed --compile --install-headers /home/solver/venv/include/site/python3.7/Cython]",[cwd: /tmp/pip-install-bnnj141r/Cython/],"[Complete output (343 lines):, Unable to find pgen, not compiling formal grammar., running bdist_wheel, running build, running build_py, creating build, creating build/lib.linux-x86_64-3.7, copying cython.py -> build/lib.linux-x86_64-3.7, creating build/lib.linux-x86_64-3.7/Cython, copying Cython/__init__.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/Utils.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/TestUtils.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/StringIOTree.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/Shadow.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/Debugging.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/Coverage.py -> build/lib.linux-x86_64-3.7/Cython, copying Cython/CodeWriter.py -> build/lib.linux-x86_64-3.7/Cython, creating build/lib.linux-x86_64-3.7/Cython/Build, copying Cython/Build/__init__.py -> build/lib.linux-x86_64-3.7/Cython/Build, copying Cython

In [42]:
error_df.to_csv('error-clean-data.csv', index=False)

In [43]:
len(error_df)

19219